In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pickle

use_pca = True
use_lda = True

if use_pca and not use_lda:
    print('using pca model')
    X_train = np.load(',/dimension_reduced_data/X_train_pca.npy')
    X_val = np.load('/dimension_reduced_data/X_val_pca.npy')
    # with open('/dimension_reduced_data/pca_model.pkl', 'rb') as f:
    #     trained_model = pickle.load(f)

if use_lda and not use_pca:
    print('using lda model')
    X_train = np.load('./dimension_reduced_data/X_train_lda.npy')
    X_val = np.load('/dimension_reduced_data/X_val_lda.npy')
    # with open('/dimension_reduced_data/lda_model.pkl', 'rb') as f:
    #     trained_model = pickle.load(f)

if use_pca and use_lda:
    print('using pca and lda model')
    X_train = np.load('./dimension_reduced_data/X_train_pca_lda.npy')
    X_val = np.load('./dimension_reduced_data/X_val_pca_lda.npy')
    # with open('/dimension_reduced_data/pca_lda_model.pkl', 'rb') as f:
    #     trained_model = pickle.load(f)


print(X_train.shape, X_val.shape)

best_accuracy_validation = 0
best_n_neighbors_validation = 1

best_accuracy_training = 0
best_n_neighbors_training = 1
neighbors_range = range(1,100)

y_train = np.load('dimension_reduced_data/y_train.npy')
y_val = np.load('dimension_reduced_data/y_val.npy')

for n_neighbors in neighbors_range:
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(X_train, y_train)

    yp = model.predict(X_train)
    training_accuracy = accuracy_score(y_train, yp)

    yp = model.predict(X_val)
    validation_accuracy = accuracy_score(y_val, yp)

    if training_accuracy > best_accuracy_training:
        best_accuracy_training = training_accuracy
        best_n_neighbors_training = n_neighbors

    if validation_accuracy > best_accuracy_validation:
        best_accuracy_validation = validation_accuracy
        best_n_neighbors_validation = n_neighbors

    print("Accuracy:",training_accuracy, validation_accuracy)

print("Best Accuracy:",best_accuracy_training, best_n_neighbors_training)
print("Best Accuracy:",best_accuracy_validation, best_n_neighbors_validation)



using pca and lda model
(10601, 32) (150, 32)


NameError: name 'y' is not defined